<a href="https://colab.research.google.com/github/neginsadat/EA/blob/main/Another_copy_of_untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!apt-get update
!apt-get install -y python3-rna



Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
import RNA
print("ViennaRNA version:", RNA.__version__)


ViennaRNA version: 2.7.0


In [ ]:
!pip uninstall viennarna
!pip install viennarna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 4.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import RNA
print("ViennaRNA version:", RNA.__version__)

ViennaRNA version: 2.7.0


In [ ]:
!ls /content/drive/MyDrive/


 46ae9c2a-8058-4cef-b7e1-e613a2c2f89c.gdoc
 chatsettingsbackup.db.crypt1
'Colab Notebooks'
'Google AI Studio'
 SmsContactsBackup
 statusranking.db.crypt1
 stickers.db.crypt1
'To-do list.gsheet'
'Untitled presentation.gslides'
'این رو به فارسی ترجمه کن و جدول آن رو بکش چیز دیگ....gsheet'
'این صفحه رو ترجمه و توضیح بده.gsheet'
'این عکس را  با دقیق  ترین حالت ممکن ترجمه کن.gsheet'
'به نام خدای بخشنده ی مهربان.gdoc'
 تحقیق.gslides
'جدید '
'جلسه اول رباتیک .gdoc'
'دانلود مقاله طراحی سیستمهای مدرن اطلاعاتی و بازشناسی خودكار الگوها.gdoc'
'سند بی‌عنوان (1).gdoc'
'سند بی‌عنوان (2).gdoc'
'سند بی‌عنوان (3).gdoc'
'سند بی‌عنوان.gdoc'
'کپی از Retro Internet Interface Personal Organizer for College by Slidesgo.gslides'


In [ ]:
import random
import time
import math
from typing import List, Tuple
import sys
import RNA

#############################################################################
# Updated Parameters (Sample values; adjust as needed)
#############################################################################

POP_SIZE = 80            # Increased population size from 52 to 80
MAX_TIME = 24 * 3600     # 24 hours total (for long runs)
SBX_PROB = 0.9
ETA_C = 10
ETA_M = 5
MUT_PROB = 1e-2          # Mutation rate for m2dRNAs

STAGNATION_GEN_ES = 5
MUT_PROB_ES = 0.05      # Increased ES mutation rate
TIME_PER_CALL_ES = 20.0  # Increased ES call time from 2 sec to 20 sec

#############################################################################
# Functions for dot-bracket structure processing
#############################################################################

def parse_dot_bracket(db: str) -> List[Tuple[int, int]]:
    stack = []
    pairs = []
    for i, ch in enumerate(db):
        if ch == '(':
            stack.append(i)
        elif ch == ')':
            if stack:
                open_i = stack.pop()
                pairs.append((open_i, i))
    pairs.sort(key=lambda x: x[0])
    return pairs

def similarity_db(pred: str, target: str) -> float:
    if len(pred) != len(target):
        return 0.0
    match_count = sum(1 for i in range(len(pred)) if pred[i] == target[i])
    return match_count / len(target)

#############################################################################
# RNA folding functions using PyViennaRNA
#############################################################################

def fold_rna(seq: str) -> str:
    """Compute the MFE secondary structure using ViennaRNA."""
    fc = RNA.fold_compound(seq)
    ss, mfe = fc.mfe()
    return ss

def partition_function(seq: str) -> float:
    """
    Compute an approximate partition function indicator using free energy.
    Uses fc.pf() then fc.mfe() to get the free energy (mfe) and converts it via e^(-ΔG/RT).
    """
    fc = RNA.fold_compound(seq)
    fc.pf()  # Compute partition function internally
    (ss, mfe) = fc.mfe()  # Get MFE and structure
    free_energy = mfe     # free_energy is given by mfe (a negative number)
    return math.exp(-free_energy / 0.61633)  # Approximation (R*T ~ 0.61633 at 37°C)

def ensemble_diversity(seq: str) -> float:
    """
    Compute the ensemble diversity using fc.mean_bp_distance().
    """
    fc = RNA.fold_compound(seq)
    fc.pf()
    diversity = fc.mean_bp_distance()
    return diversity

#############################################################################
# Composition objective function
#############################################################################

def composition_objective(seq: str) -> float:
    """
    Calculate the standard deviation of nucleotide frequencies as a measure of composition uniformity.
    Lower values indicate a more balanced distribution.
    """
    n = len(seq)
    cntA = seq.count('A')
    cntC = seq.count('C')
    cntG = seq.count('G')
    cntU = seq.count('U')
    pA = cntA / n
    pC = cntC / n
    pG = cntG / n
    pU = cntU / n
    mean_ = (pA + pC + pG + pU) / 4
    variance = ((pA - mean_)**2 + (pC - mean_)**2 + (pG - mean_)**2 + (pU - mean_)**2) / 4
    std_ = math.sqrt(variance)
    return std_

#############################################################################
# Chromosome decoding
#############################################################################

def decode_chromosome(chrom: list, pairs: List[Tuple[int, int]], length_db: int) -> str:
    """
    Convert the chromosome (a list of real values) to an RNA sequence.
    The first part of the chromosome (length = len(pairs)) is used to assign base pairs,
    and the remainder is used for unpaired positions.
    """
    n_pairs = len(pairs)
    n_unpaired = length_db - 2 * n_pairs
    if n_unpaired < 0:
        n_unpaired = 0

    seq_list = ['A'] * length_db
    # For paired positions:
    for i, (a, b) in enumerate(pairs):
        val = chrom[i]
        if val < 0.33:
            seq_list[a] = 'G'
            seq_list[b] = 'C'
        elif val < 0.66:
            seq_list[a] = 'A'
            seq_list[b] = 'U'
        else:
            seq_list[a] = 'G'
            seq_list[b] = 'U'

    # For unpaired positions:
    up_chrom = chrom[n_pairs:]
    up_idx = 0
    for i in range(length_db):
        if seq_list[i] == 'A':  # If still unassigned (default 'A')
            if up_idx < len(up_chrom):
                val = up_chrom[up_idx]
                up_idx += 1
                if val < 0.25:
                    seq_list[i] = 'A'
                elif val < 0.5:
                    seq_list[i] = 'C'
                elif val < 0.75:
                    seq_list[i] = 'G'
                else:
                    seq_list[i] = 'U'
    return "".join(seq_list)

#############################################################################
# NSGA-II Functions
#############################################################################

def sbx_crossover(p1: list, p2: list, eta_c: float = ETA_C, pc: float = SBX_PROB):
    size = len(p1)
    if random.random() > pc:
        return p1[:], p2[:]
    c1, c2 = p1[:], p2[:]
    for i in range(size):
        if random.random() < 0.5:
            if abs(p1[i] - p2[i]) > 1e-14:
                y1 = min(p1[i], p2[i])
                y2 = max(p1[i], p2[i])
                rand_ = random.random()
                beta = 1.0 + (2.0 * y1 / (y2 - y1))
                alpha = 2.0 - pow(beta, -(eta_c + 1.0))
                if rand_ <= 1.0 / alpha:
                    betaq = pow(rand_ * alpha, 1.0 / (eta_c + 1.0))
                else:
                    betaq = pow(1.0 / (2.0 - rand_ * alpha), 1.0 / (eta_c + 1.0))
                c1_i = 0.5 * ((y1 + y2) - betaq * (y2 - y1))

                beta = 1.0 + (2.0 * (1.0 - y2) / (y2 - y1))
                alpha = 2.0 - pow(beta, -(eta_c + 1.0))
                if rand_ <= 1.0 / alpha:
                    betaq = pow(rand_ * alpha, 1.0 / (eta_c + 1.0))
                else:
                    betaq = pow(1.0 / (2.0 - rand_ * alpha), 1.0 / (eta_c + 1.0))
                c2_i = 0.5 * ((y1 + y2) + betaq * (y2 - y1))

                c1_i = max(min(c1_i, 1.0), 0.0)
                c2_i = max(min(c2_i, 1.0), 0.0)

                if random.random() < 0.5:
                    c1[i] = c1_i
                    c2[i] = c2_i
                else:
                    c1[i] = c2_i
                    c2[i] = c1_i
    return c1, c2

def polynomial_mutation(chrom: list, eta_m: float = ETA_M):
    for i in range(len(chrom)):
        if random.random() < (1.0 / len(chrom)):
            y = chrom[i]
            delta1 = (y - 0.0) / (1.0 - 0.0)
            delta2 = (1.0 - y) / (1.0 - 0.0)
            rand_ = random.random()
            mut_pow = 1.0 / (eta_m + 1.0)
            if rand_ < 0.5:
                xy = 1.0 - delta1
                val = 2.0 * rand_ + (1.0 - 2.0 * rand_) * pow(xy, (eta_m + 1.0))
                deltaq = pow(val, mut_pow) - 1.0
            else:
                xy = 1.0 - delta2
                val = 2.0 * (1.0 - rand_) + 2.0 * (rand_ - 0.5) * pow(xy, (eta_m + 1.0))
                deltaq = 1.0 - pow(val, mut_pow)
            y = y + deltaq
            y = max(min(y, 1.0), 0.0)
            chrom[i] = y

def fast_non_dominated_sort(pop_objs: List[List[float]]) -> List[List[int]]:
    pop_size = len(pop_objs)
    S = [[] for _ in range(pop_size)]
    n = [0] * pop_size
    rank = [0] * pop_size
    fronts = [[]]

    for p in range(pop_size):
        S[p] = []
        n[p] = 0
        for q in range(pop_size):
            p_better = False
            q_better = False
            for k in range(len(pop_objs[p])):
                if pop_objs[p][k] < pop_objs[q][k]:
                    q_better = True
                elif pop_objs[p][k] > pop_objs[q][k]:
                    p_better = True
            if (q_better and not p_better):
                n[p] += 1
            elif (p_better and not q_better):
                S[p].append(q)
        if n[p] == 0:
            rank[p] = 0
            fronts[0].append(p)

    i = 0
    while len(fronts[i]) > 0:
        next_front = []
        for p in fronts[i]:
            for q in S[p]:
                n[q] -= 1
                if n[q] == 0:
                    rank[q] = i + 1
                    next_front.append(q)
        i += 1
        fronts.append(next_front)
    while len(fronts[-1]) == 0:
        fronts.pop()
        if not fronts:
            break
    return fronts

#############################################################################
# m2dRNAs (Simplified version with NSGA-II)
#############################################################################

def m2dRNAs(target_db: str, max_time: float) -> Tuple[str, str]:
    """
    Uses a simplified NSGA-II with three objective functions:
      1) partition function
      2) ensemble diversity
      3) composition
    Returns a sequence whose predicted structure approaches the target.
    """
    start_time = time.time()

    pairs = parse_dot_bracket(target_db)
    length_db = len(target_db)
    n_pairs = len(pairs)
    n_unp = length_db - 2 * n_pairs
    if n_unp < 0:
        n_unp = 0
    chrom_len = n_pairs + n_unp

    population = []
    for _ in range(POP_SIZE):
        ind = [random.random() for __ in range(chrom_len)]
        population.append(ind)

    def evaluate_population(pop):
        obj_values = []
        seq_cache = []
        for indiv in pop:
            seq_ = decode_chromosome(indiv, pairs, length_db)
            pf = partition_function(seq_)
            ed = ensemble_diversity(seq_)
            co = composition_objective(seq_)
            obj_values.append([pf, ed, co])
            seq_cache.append(seq_)
        return obj_values, seq_cache

    generation = 0
    while True:
        elapsed = time.time() - start_time
        if elapsed >= max_time:
            break

        obj_vals, seq_cache = evaluate_population(population)
        for i, seq_ in enumerate(seq_cache):
            pred_db = fold_rna(seq_)
            if pred_db == target_db:
                return (seq_, pred_db)

        fronts = fast_non_dominated_sort(obj_vals)
        new_pop = []
        for front in fronts:
            if len(new_pop) + len(front) <= POP_SIZE:
                for idx_ in front:
                    new_pop.append(population[idx_])
            else:
                needed = POP_SIZE - len(new_pop)
                random.shuffle(front)
                for idx_ in front[:needed]:
                    new_pop.append(population[idx_])
                break
        while len(new_pop) < POP_SIZE:
            new_pop.append(population[0])

        offspring = []
        for i in range(0, POP_SIZE, 2):
            p1 = new_pop[i]
            p2 = new_pop[i+1] if i+1 < POP_SIZE else new_pop[0]
            c1, c2 = sbx_crossover(p1, p2)
            polynomial_mutation(c1)
            polynomial_mutation(c2)
            offspring.append(c1)
            offspring.append(c2)

        population = offspring[:POP_SIZE]
        generation += 1
        if generation > 10000:
            break

    obj_vals, seq_cache = evaluate_population(population)
    fronts = fast_non_dominated_sort(obj_vals)
    best_ind = fronts[0][0]
    best_seq = seq_cache[best_ind]
    best_pred = fold_rna(best_seq)
    return (best_seq, best_pred)

#############################################################################
# eM2dRNAs (Simplified)
#############################################################################

def eM2dRNAs(target_db: str, guide: List[int], max_time: float) -> Tuple[str, str]:
    # In this simplified version, we simply call m2dRNAs once.
    seq, pred_db = m2dRNAs(target_db, max_time)
    return seq, pred_db

#############################################################################
# ES (Evolutionary Strategy)
#############################################################################

def es_em2dRNAs(target_db: str, total_time: float) -> Tuple[str, str]:
    start_time = time.time()
    pairs = parse_dot_bracket(target_db)
    es_budget = total_time / 2.0  # allocate half time for ES
    def greedy_guide():
        g = [0] * len(pairs)
        for i in range(len(g)):
            if i % 10 == 0:
                g[i] = 1
        return g
    x = greedy_guide()
    Xx, Sx = eM2dRNAs(target_db, x, TIME_PER_CALL_ES)
    best_sim = similarity_db(Sx, target_db)
    pm_es = MUT_PROB_ES
    stagnation = 0
    while True:
        elapsed = time.time() - start_time
        if elapsed >= es_budget:
            break
        if best_sim >= 1.0:
            break
        def mutate_guide(guide, pm):
            new_g = guide[:]
            for i in range(len(new_g)):
                if random.random() < pm:
                    new_g[i] = 1 - new_g[i]
            return new_g
        y = mutate_guide(x, pm_es)
        Xy, Sy = eM2dRNAs(target_db, y, TIME_PER_CALL_ES)
        sim_y = similarity_db(Sy, target_db)
        if sim_y > best_sim:
            x = y
            Xx, Sx = Xy, Sy
            best_sim = sim_y
            pm_es = MUT_PROB_ES
            stagnation = 0
        else:
            stagnation += 1
            if stagnation >= STAGNATION_GEN_ES:
                pm_es *= 1.2
                stagnation = 0
    used = time.time() - start_time
    remain = total_time - used
    if remain > 0 and best_sim < 1.0:
        XX, SS = eM2dRNAs(target_db, x, remain)
        sim_ = similarity_db(SS, target_db)
        if sim_ > best_sim:
            Xx, Sx = XX, SS
            best_sim = sim_
    return (Xx, Sx)

#############################################################################
# Read Eterna Dataset
#############################################################################

def read_eterna_dataset(filename: str):
    puzzles = []
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line.startswith("Puzzle #"):
                continue
            fields = line.split('\t')
            if len(fields) < 5:
                continue
            puzzle_id = fields[0].strip()
            puzzle_name = fields[1].strip()
            secondary_structure = fields[4].strip()
            puzzles.append((puzzle_id, puzzle_name, secondary_structure))
    return puzzles

#############################################################################
# Main Function
#############################################################################

def main():
    random.seed(0)
    dataset_file = "/content/drive/MyDrive/Colab Notebooks/eterna100_vienna2.txt"
    puzzles = read_eterna_dataset(dataset_file)
    print(f"Read {len(puzzles)} puzzles from {dataset_file}")

    # Increase time per puzzle to 600 seconds (10 minutes)
    time_per_puzzle = 600

    for idx, (pid, pname, pdb) in enumerate(puzzles):
        print(f"\nSolving puzzle #{pid} ({pname}) ...")
        seq, pred = es_em2dRNAs(pdb, time_per_puzzle)
        sim = similarity_db(pred, pdb)
        print(f"  Found seq = {seq}")
        print(f"  Pred DB   = {pred}")
        print(f"  Similarity= {sim:.4f}")

if __name__ == "__main__":
    main()


Read 100 puzzles from /content/drive/MyDrive/Colab Notebooks/eterna100_vienna2.txt

Solving puzzle #1 (Simple Hairpin) ...
  Found seq = GGGGGCAGAAACCCUC
  Pred DB   = (((((......)))))
  Similarity= 1.0000

Solving puzzle #2 (Arabidopsis Thaliana 6 RNA - Difficulty Level 0) ...
  Found seq = GGGGGGGCCAUGGGGUUUGUGCCCUUUCGGGGGGGGUUGGGGGGGGGAAGGGGGGGGCGCAGAGGUGAAAACUUCUACUCUCUAUCUUUUUUUCCUCAACUUUCCUCAACCCUCUU
  Pred DB   = .((((((....(((((....)))))...(((((((((((((((((((((((((((((...((((((....)))))).)))))).)))))))))))))))))))))))..)))))).
  Similarity= 0.8448

Solving puzzle #3 (Prion Pseudoknot - Difficulty Level 0) ...
  Found seq = GGGAGGAGGAGUGAACUCCCCUACCUCACGCCAGAA
  Pred DB   = ((((((.((((....))))))).)))..........
  Similarity= 1.0000

Solving puzzle #4 (Human Integrated Adenovirus - Difficulty Level 0) ...
  Found seq = GGGGGGGGGGGCGGAGGUGAGUGGAUAGCGCGGGGGGUGGGAAGCAAGCUGGGGGGGCAAUGGGGUGGGGGGGGCUAGCGGCCUCCUUUUUUGGUUUUUUUCUUCUGGUUUUUUCCUUUCUUUCUUGCGGGGGUCGGCUCUCCUUUUUCACCGUUGUAUCAUUU

KeyboardInterrupt: 

In [ ]:
import random
import time
import math
from typing import List, Tuple
import sys
import RNA
import concurrent.futures

#############################################################################
# تنظیمات کلی
#############################################################################

POP_SIZE = 52            # اندازه جمعیت
SBX_PROB = 0.9
ETA_C = 10
ETA_M = 5
MUT_PROB = 0.01          # نرخ جهش پایه

STAGNATION_GEN_ES = 5
MUT_PROB_ES = 0.02
TIME_PER_CALL_ES = 10.0  # زمان فراخوان ES (به ثانیه)

MAX_GENERATION = 200  # حداکثر نسل در m2dRNAs

#############################################################################
# توابع مربوط به ساختار dot-bracket
#############################################################################

def parse_dot_bracket(db: str) -> List[Tuple[int,int]]:
    stack = []
    pairs = []
    for i, ch in enumerate(db):
        if ch == '(':
            stack.append(i)
        elif ch == ')':
            if stack:
                open_i = stack.pop()
                pairs.append((open_i, i))
    pairs.sort(key=lambda x: x[0])
    return pairs

def similarity_db(pred: str, target: str) -> float:
    if len(pred) != len(target):
        return 0.0
    match_count = sum(1 for i in range(len(pred)) if pred[i] == target[i])
    return match_count / len(target)

#############################################################################
# توابع فولد RNA
#############################################################################

def fold_rna(seq: str) -> str:
    fc = RNA.fold_compound(seq)
    ss, mfe = fc.mfe()
    return ss

def partition_function(seq: str) -> float:
    fc = RNA.fold_compound(seq)
    fc.pf()
    (ss, mfe) = fc.mfe()
    free_energy = mfe
    return math.exp(-free_energy / 0.61633)

def ensemble_diversity(seq: str) -> float:
    fc = RNA.fold_compound(seq)
    fc.pf()
    return fc.mean_bp_distance()

def composition_objective(seq: str) -> float:
    n = len(seq)
    pA = seq.count('A') / n
    pC = seq.count('C') / n
    pG = seq.count('G') / n
    pU = seq.count('U') / n
    mean_ = (pA + pC + pG + pU) / 4
    variance = ((pA - mean_)**2 + (pC - mean_)**2 +
                (pG - mean_)**2 + (pU - mean_)**2) / 4
    return math.sqrt(variance)

#############################################################################
# decode کروموزوم
#############################################################################

def decode_chromosome(chrom: list, pairs: List[Tuple[int,int]], length_db: int) -> str:
    n_pairs = len(pairs)
    n_unp = length_db - 2*n_pairs
    if n_unp < 0:
        n_unp = 0
    seq_list = ['A']*length_db
    for i, (a,b) in enumerate(pairs):
        val = chrom[i]
        if val < 0.33:
            seq_list[a] = 'G'
            seq_list[b] = 'C'
        elif val < 0.66:
            seq_list[a] = 'A'
            seq_list[b] = 'U'
        else:
            seq_list[a] = 'G'
            seq_list[b] = 'U'
    up_chrom = chrom[n_pairs:]
    idx_ = 0
    for i in range(length_db):
        if seq_list[i] == 'A':  # پیش‌فرض
            if idx_ < len(up_chrom):
                val = up_chrom[idx_]
                idx_ += 1
                if val < 0.25:
                    seq_list[i] = 'A'
                elif val < 0.5:
                    seq_list[i] = 'C'
                elif val < 0.75:
                    seq_list[i] = 'G'
                else:
                    seq_list[i] = 'U'
    return "".join(seq_list)

#############################################################################
# SBX و Mut
#############################################################################

def sbx_crossover(p1: list, p2: list, eta_c: float=ETA_C, pc: float=SBX_PROB):
    size = len(p1)
    if random.random() > pc:
        return p1[:], p2[:]
    c1, c2 = p1[:], p2[:]
    for i in range(size):
        if random.random() < 0.5:
            if abs(p1[i] - p2[i])>1e-14:
                y1 = min(p1[i], p2[i])
                y2 = max(p1[i], p2[i])
                rand_ = random.random()
                beta = 1.0 + (2.0*(y1)/(y2-y1))
                alpha = 2.0 - (beta**(-(eta_c+1.0)))
                if rand_ <= 1.0/alpha:
                    betaq = (rand_*alpha)**(1.0/(eta_c+1.0))
                else:
                    betaq = (1.0/(2.0 - rand_*alpha))**(1.0/(eta_c+1.0))
                c1_i = 0.5*((y1+y2) - betaq*(y2-y1))
                beta = 1.0 + (2.0*(1.0-y2)/(y2-y1))
                alpha = 2.0 - (beta**(-(eta_c+1.0)))
                if rand_ <= 1.0/alpha:
                    betaq = (rand_*alpha)**(1.0/(eta_c+1.0))
                else:
                    betaq = (1.0/(2.0 - rand_*alpha))**(1.0/(eta_c+1.0))
                c2_i = 0.5*((y1+y2) + betaq*(y2-y1))
                c1_i = max(min(c1_i, 1.0), 0.0)
                c2_i = max(min(c2_i, 1.0), 0.0)
                if random.random() < 0.5:
                    c1[i] = c1_i
                    c2[i] = c2_i
                else:
                    c1[i] = c2_i
                    c2[i] = c1_i
    return c1, c2

def polynomial_mutation(chrom: list, gen: int, total_gen: int, base_mut=MUT_PROB):
    """Mutation با در نظر گرفتن intensification بعد از نصف نسل"""
    if gen > total_gen//2:
        base_mut *= 1.5  # کمی افزایش نرخ جهش در نیمه دوم
    for i in range(len(chrom)):
        if random.random() < base_mut:
            y = chrom[i]
            delta1 = (y-0.0)/(1.0-0.0)
            delta2 = (1.0-y)/(1.0-0.0)
            rand_ = random.random()
            mut_pow = 1.0/(ETA_M+1.0)
            if rand_ < 0.5:
                xy = 1.0 - delta1
                val = 2.0*rand_ + (1.0 - 2.0*rand_)*xy**(ETA_M+1.0)
                deltaq = val**(mut_pow) - 1.0
            else:
                xy = 1.0 - delta2
                val = 2.0*(1.0 - rand_) + 2.0*(rand_ - 0.5)*xy**(ETA_M+1.0)
                deltaq = 1.0 - val**(mut_pow)
            y = y + deltaq
            y = max(min(y, 1.0), 0.0)
            chrom[i] = y

#############################################################################
# Non-dominated Sort
#############################################################################

def fast_non_dominated_sort(obj_vals: List[List[float]]) -> List[List[int]]:
    pop_size = len(obj_vals)
    S = [[] for _ in range(pop_size)]
    n = [0]*pop_size
    rank = [0]*pop_size
    fronts = [[]]

    for p in range(pop_size):
        S[p] = []
        n[p] = 0
        for q in range(pop_size):
            p_better = False
            q_better = False
            for k in range(len(obj_vals[p])):
                if obj_vals[p][k] < obj_vals[q][k]:
                    q_better = True
                elif obj_vals[p][k] > obj_vals[q][k]:
                    p_better = True
            if q_better and not p_better:
                n[p]+=1
            elif p_better and not q_better:
                S[p].append(q)
        if n[p] == 0:
            rank[p] = 0
            fronts[0].append(p)

    i = 0
    while len(fronts[i])>0:
        next_front = []
        for p in fronts[i]:
            for q in S[p]:
                n[q] -=1
                if n[q] == 0:
                    rank[q] = i+1
                    next_front.append(q)
        i+=1
        fronts.append(next_front)
    while len(fronts[-1])==0:
        fronts.pop()
        if not fronts:
            break
    return fronts

#############################################################################
# ارزیابی موازی
#############################################################################

def evaluate_individual(indiv: list, pairs, length_db):
    seq_ = decode_chromosome(indiv, pairs, length_db)
    pf = partition_function(seq_)
    ed = ensemble_diversity(seq_)
    co = composition_objective(seq_)
    return [pf, ed, co], seq_

#############################################################################
# m2dRNAs با multi-run کوتاه
#############################################################################

def run_nsga(target_db: str, max_time: float) -> Tuple[str,str]:
    """یک اجرای NSGA-II در زمان مشخص"""
    start_time = time.time()

    pairs = parse_dot_bracket(target_db)
    length_db = len(target_db)
    n_pairs = len(pairs)
    n_unp = length_db - 2*n_pairs
    if n_unp<0:
        n_unp = 0
    chrom_len = n_pairs + n_unp

    # مقداردهی اولیه
    population = []
    for _ in range(POP_SIZE):
        # مقداردهی اولیه تصادفی
        ind = [random.random() for __ in range(chrom_len)]
        population.append(ind)

    generation = 0
    while True:
        if time.time()-start_time >= max_time:
            break
        # ارزیابی موازی
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = [executor.submit(evaluate_individual, pop, pairs, length_db)
                       for pop in population]
            results = [f.result() for f in concurrent.futures.as_completed(futures)]

        obj_values = []
        seq_cache = []
        idx_ = 0
        for (ov, sq) in results:
            obj_values.append(ov)
            seq_cache.append(sq)
            idx_+=1

        # بررسی شباهت=1 (Early Stopping)
        for i, sq_ in enumerate(seq_cache):
            if fold_rna(sq_) == target_db:
                return (sq_, target_db)

        # Fast non-dominated sort
        fronts = fast_non_dominated_sort(obj_values)
        new_pop = []
        for fr in fronts:
            if len(new_pop)+len(fr)<=POP_SIZE:
                for idx in fr:
                    new_pop.append(population[idx])
            else:
                needed = POP_SIZE - len(new_pop)
                random.shuffle(fr)
                new_pop.extend([population[idx] for idx in fr[:needed]])
                break
        while len(new_pop) < POP_SIZE:
            new_pop.append(population[0])

        # تولید offspring
        offspring = []
        for i in range(0, POP_SIZE, 2):
            p1 = new_pop[i]
            p2 = new_pop[i+1] if i+1<POP_SIZE else new_pop[0]
            c1, c2 = sbx_crossover(p1, p2)
            polynomial_mutation(c1, generation, MAX_GENERATION, base_mut=MUT_PROB)
            polynomial_mutation(c2, generation, MAX_GENERATION, base_mut=MUT_PROB)
            offspring.append(c1)
            offspring.append(c2)
        population = offspring[:POP_SIZE]

        generation +=1
        if generation>MAX_GENERATION:
            break

    # در انتها بهترین را برگردانیم
    best_idx=0
    best_sim=0
    for i, pop_ in enumerate(population):
        sq_= decode_chromosome(pop_, pairs, length_db)
        sim_ = similarity_db(fold_rna(sq_), target_db)
        if sim_>best_sim:
            best_sim=sim_
            best_idx=i
    final_seq = decode_chromosome(population[best_idx], pairs, length_db)
    final_pred = fold_rna(final_seq)
    return (final_seq, final_pred)

def multi_run_short(target_db: str, run_times: int, each_time: float) -> Tuple[str,str,float]:
    """چندبار اجرای کوتاه انجام بده و بهترین را گزارش کن."""
    best_seq = None
    best_db = ""
    best_sim = 0.0
    for _ in range(run_times):
        seq_, pred_ = run_nsga(target_db, each_time)
        sim_ = similarity_db(pred_, target_db)
        if sim_>best_sim:
            best_sim=sim_
            best_seq=seq_
            best_db=pred_
        # اگر به ساختار کامل رسید
        if best_sim>=1.0:
            break
    return (best_seq, best_db, best_sim)

#############################################################################
# ES (کمی ساده)
#############################################################################

def es_em2dRNAs(target_db: str, total_time: float) -> Tuple[str,str]:
    """
    برای سادگی، فقط از multi_run_short بجای ساختار ES اصلی استفاده میکنیم
    یا میتوانیم همین را با ES ترکیب کنیم، فعلاً ساده می‌کنیم:
    """
    # بجای پیاده‌سازی ES کامل، صرفاً multi-run را انجام می‌دهیم
    # تا نتایج سریع‌تر به جواب برسد
    runs = 5
    each_time = total_time/runs
    seq_, pred_, sim_ = multi_run_short(target_db, runs, each_time)
    return (seq_, pred_)

#############################################################################
# خواندن دیتاست
#############################################################################

def read_eterna_dataset(filename: str):
    puzzles = []
    with open(filename, 'r') as f:
        for line in f:
            line=line.strip()
            if not line or line.startswith("Puzzle #"):
                continue
            fields = line.split('\t')
            if len(fields)<5:
                continue
            puzzle_id = fields[0].strip()
            puzzle_name = fields[1].strip()
            secondary_structure = fields[4].strip()
            puzzles.append((puzzle_id, puzzle_name, secondary_structure))
    return puzzles

#############################################################################
# main
#############################################################################

def main():
    random.seed(0)
    dataset_file = "/content/drive/MyDrive/Colab Notebooks/eterna100_vienna2.txt"
    puzzles = read_eterna_dataset(dataset_file)
    print(f"Read {len(puzzles)} puzzles from {dataset_file}")

    # فقط 10 پازل اول را حل می‌کنیم (برای کمبود زمان)
    puzzles = puzzles[:10]

    # هر پازل را با multi-run کوتاه (مثلاً 60 ثانیه) حل می‌کنیم
    time_per_puzzle = 600.0

    for idx, (pid, pname, pdb) in enumerate(puzzles, start=1):
        print(f"\nSolving puzzle #{pid} ({pname}) ...")
        seq, pred = es_em2dRNAs(pdb, time_per_puzzle)
        sim = similarity_db(pred, pdb)
        print(f"  Found seq = {seq}")
        print(f"  Pred DB   = {pred}")
        print(f"  Similarity= {sim:.4f}")

if __name__=="__main__":
    main()


Read 100 puzzles from /content/drive/MyDrive/Colab Notebooks/eterna100_vienna2.txt

Solving puzzle #1 (Simple Hairpin) ...
  Found seq = GGGGGCAGAAACCCUC
  Pred DB   = (((((......)))))
  Similarity= 1.0000

Solving puzzle #2 (Arabidopsis Thaliana 6 RNA - Difficulty Level 0) ...
  Found seq = GGAAGGGCCUGGGGAGUAGCGAACCCCUGGGGAGGAGGGGGGGGGGAGGUGGGGGAGCCGGGGGGAGGAAAUUCUCAUUUCUCACUUUUCUCUCUCCAAUCUUCCUUAAUCCUUUC
  Pred DB   = (((((((...((((.........))))(((((((((((((((((((((((((((((...(((((.......))))).))))))))))))))))))))..))))))))).)))))))
  Similarity= 0.9052

Solving puzzle #3 (Prion Pseudoknot - Difficulty Level 0) ...
  Found seq = GAGGGGUGGGAUUGAUCCCCCCGCUCUUUUCCAAAA
  Pred DB   = ((((((.((((....))))))).)))..........
  Similarity= 1.0000

Solving puzzle #4 (Human Integrated Adenovirus - Difficulty Level 0) ...
  Found seq = UAGCGGGGGGGUAUGGGGAGGCAAUGGAAUAUGGCGGGGGGAGGCGUAUGGGAGGGUAUACGGGUGGGGAAGGGGAUGAAUACUUUUUCCUUAUCAUUUUCCCUUCAGCUUUCUGACUCUCCUGCAGGGGGGUUACUUUCCUUCCUUUUAUCAAAAAAAAACCU

In [1]:
import random
import time
import math
from typing import List, Tuple
import sys
import RNA

#############################################################################
# پارامترها مطابق مقادیر رایج در مقاله
#############################################################################

POP_SIZE = 52         # اندازه جمعیت
SBX_PROB = 0.9        # احتمال کراس‌اور (SBX)
ETA_C = 10            # پارامتر SBX
ETA_M = 5             # پارامتر Polynomial Mutation
STAGNATION_GEN_ES = 5 # تعداد نسل رکود در ES
MUT_PROB_ES = 0.01    # نرخ جهش ES
TIME_PER_CALL_ES = 2.0 # برای ES

#############################################################################
# توابع مربوط به ساختار dot-bracket
#############################################################################

def parse_dot_bracket(db: str) -> List[Tuple[int,int]]:
    stack = []
    pairs = []
    for i, ch in enumerate(db):
        if ch == '(':
            stack.append(i)
        elif ch == ')':
            if stack:
                open_i = stack.pop()
                pairs.append((open_i, i))
    pairs.sort(key=lambda x: x[0])
    return pairs

def similarity_db(pred: str, target: str) -> float:
    if len(pred) != len(target):
        return 0.0
    match_count = sum(1 for i in range(len(pred)) if pred[i] == target[i])
    return match_count / len(target)

#############################################################################
# توابع فولد RNA با PyViennaRNA
#############################################################################

def fold_rna(seq: str) -> str:
    fc = RNA.fold_compound(seq)
    ss, mfe = fc.mfe()
    return ss

def partition_function(seq: str) -> float:
    fc = RNA.fold_compound(seq)
    fc.pf()  # محاسبه پارتیشن فانکشن
    (ss, mfe) = fc.mfe()
    # طبق مقاله، از e^(-ΔG/RT) استفاده می‌کنیم
    return math.exp(-mfe / 0.61633)  # تقریب RT در 37 درجه

def ensemble_diversity(seq: str) -> float:
    fc = RNA.fold_compound(seq)
    fc.pf()
    return fc.mean_bp_distance()

def composition_objective(seq: str) -> float:
    n = len(seq)
    pA = seq.count('A')/n
    pC = seq.count('C')/n
    pG = seq.count('G')/n
    pU = seq.count('U')/n
    mean_ = (pA + pC + pG + pU)/4
    var_ = ((pA-mean_)**2 + (pC-mean_)**2 + (pG-mean_)**2 + (pU-mean_)**2)/4
    return math.sqrt(var_)

#############################################################################
# decode
#############################################################################

def decode_chromosome(chrom: list, pairs: List[Tuple[int,int]], length_db: int) -> str:
    n_pairs = len(pairs)
    n_unp = length_db - 2*n_pairs
    if n_unp<0:
        n_unp=0
    seq_list = ['A']*length_db
    for i, (a,b) in enumerate(pairs):
        val = chrom[i]
        if val < 0.33:
            seq_list[a] = 'G'
            seq_list[b] = 'C'
        elif val < 0.66:
            seq_list[a] = 'A'
            seq_list[b] = 'U'
        else:
            seq_list[a] = 'G'
            seq_list[b] = 'U'
    up_chrom = chrom[n_pairs:]
    idx_=0
    for i in range(length_db):
        if seq_list[i] == 'A': # پیشفرض A
            if idx_<len(up_chrom):
                val=up_chrom[idx_]
                idx_+=1
                if val<0.25:
                    seq_list[i] = 'A'
                elif val<0.5:
                    seq_list[i] = 'C'
                elif val<0.75:
                    seq_list[i] = 'G'
                else:
                    seq_list[i] = 'U'
    return "".join(seq_list)

#############################################################################
# SBX و MUT
#############################################################################

def sbx_crossover(p1: list, p2: list, eta_c: float=ETA_C, pc: float=SBX_PROB):
    size = len(p1)
    if random.random()>pc:
        return p1[:], p2[:]
    c1, c2 = p1[:], p2[:]
    for i in range(size):
        if random.random()<0.5:
            if abs(p1[i]-p2[i])>1e-14:
                y1 = min(p1[i], p2[i])
                y2 = max(p1[i], p2[i])
                rand_ = random.random()
                # فرمول کامل SBX
                beta = 1.0 + (2.0*(y1)/(y2-y1))
                alpha = 2.0 - beta**(-(eta_c+1.0))
                if rand_ <= 1.0/alpha:
                    betaq = (rand_*alpha)**(1.0/(eta_c+1.0))
                else:
                    betaq = (1.0/(2.0 - rand_*alpha))**(1.0/(eta_c+1.0))
                c1_i = 0.5*((y1+y2) - betaq*(y2-y1))

                beta = 1.0 + (2.0*(1.0-y2)/(y2-y1))
                alpha = 2.0 - beta**(-(eta_c+1.0))
                if rand_ <= 1.0/alpha:
                    betaq = (rand_*alpha)**(1.0/(eta_c+1.0))
                else:
                    betaq = (1.0/(2.0 - rand_*alpha))**(1.0/(eta_c+1.0))
                c2_i = 0.5*((y1+y2) + betaq*(y2-y1))
                c1_i = max(min(c1_i,1.0),0.0)
                c2_i = max(min(c2_i,1.0),0.0)
                if random.random()<0.5:
                    c1[i],c2[i] = c1_i, c2_i
                else:
                    c1[i],c2[i] = c2_i, c1_i
    return c1,c2

def polynomial_mutation(chrom: list, pm: float):
    size = len(chrom)
    for i in range(size):
        if random.random()< pm:
            y = chrom[i]
            delta1 = (y-0.0)/(1.0-0.0)
            delta2 = (1.0-y)/(1.0-0.0)
            rand_ = random.random()
            mut_pow = 1.0/(ETA_M+1.0)
            if rand_ < 0.5:
                xy = 1.0 - delta1
                val = 2.0*rand_ + (1.0 - 2.0*rand_)*(xy**(ETA_M+1.0))
                deltaq = val**mut_pow - 1.0
            else:
                xy = 1.0 - delta2
                val = 2.0*(1.0-rand_) + 2.0*(rand_ - 0.5)*(xy**(ETA_M+1.0))
                deltaq = 1.0 - val**mut_pow
            y = y + deltaq
            y = max(min(y,1.0),0.0)
            chrom[i] = y

#############################################################################
# fast_non_dominated_sort
#############################################################################

def fast_non_dominated_sort(pop_objs: List[List[float]]) -> List[List[int]]:
    pop_size = len(pop_objs)
    S = [[] for _ in range(pop_size)]
    n = [0]*pop_size
    rank = [0]*pop_size
    fronts = [[]]

    for p in range(pop_size):
        S[p] = []
        n[p] = 0
        for q in range(pop_size):
            p_better = False
            q_better = False
            for k in range(len(pop_objs[p])):
                if pop_objs[p][k]< pop_objs[q][k]:
                    q_better = True
                elif pop_objs[p][k]> pop_objs[q][k]:
                    p_better= True
            if q_better and not p_better:
                n[p]+=1
            elif p_better and not q_better:
                S[p].append(q)
        if n[p] == 0:
            rank[p] = 0
            fronts[0].append(p)

    i =0
    while len(fronts[i])>0:
        next_front=[]
        for p in fronts[i]:
            for q in S[p]:
                n[q]-=1
                if n[q] ==0:
                    rank[q] = i+1
                    next_front.append(q)
        i+=1
        fronts.append(next_front)
    while len(fronts[-1])==0:
        fronts.pop()
        if not fronts:
            break
    return fronts

#############################################################################
# m2dRNAs ساده با پارامترهای "شبیه مقاله"
#############################################################################

def m2dRNAs(target_db: str, max_time: float) -> Tuple[str,str]:
    start_time = time.time()
    pairs = parse_dot_bracket(target_db)
    length_db = len(target_db)
    n_pairs = len(pairs)
    n_unp = length_db - 2*n_pairs
    if n_unp<0:
        n_unp=0
    chrom_len = n_pairs+n_unp

    # مقداردهی اولیه
    population = []
    for _ in range(POP_SIZE):
        # نرخ جهش مقاله معمولاً pm = 1.0/chrom_len
        # ولی در init: از random سراغ 0..1
        indiv = [random.random() for __ in range(chrom_len)]
        population.append(indiv)

    def evaluate_population(pop):
        obj_vals = []
        seq_cache = []
        for indiv in pop:
            seq_ = decode_chromosome(indiv, pairs, length_db)
            pf = partition_function(seq_)
            ed = ensemble_diversity(seq_)
            co = composition_objective(seq_)
            obj_vals.append([pf, ed, co])
            seq_cache.append(seq_)
        return obj_vals, seq_cache

    generation=0
    while True:
        if time.time() - start_time >= max_time:
            break
        obj_vals, seq_cache = evaluate_population(population)

        # چک مشابهت=1
        for i, sq_ in enumerate(seq_cache):
            if fold_rna(sq_) == target_db:
                return (sq_, target_db)

        fronts = fast_non_dominated_sort(obj_vals)
        new_pop=[]
        for fr in fronts:
            if len(new_pop)+len(fr)<=POP_SIZE:
                for idx_ in fr:
                    new_pop.append(population[idx_])
            else:
                needed = POP_SIZE - len(new_pop)
                random.shuffle(fr)
                new_pop.extend([population[idx_] for idx_ in fr[:needed]])
                break

        # offspring
        offspring = []
        pm_value = 1.0/chrom_len  # نرخ جهش طبق مقاله
        for i in range(0,POP_SIZE,2):
            p1 = new_pop[i]
            p2 = new_pop[i+1] if i+1<POP_SIZE else new_pop[0]
            c1, c2 = sbx_crossover(p1,p2)
            polynomial_mutation(c1, pm_value)
            polynomial_mutation(c2, pm_value)
            offspring.append(c1)
            offspring.append(c2)
        population = offspring[:POP_SIZE]
        generation+=1
        if generation>10000: # یا هر مقدار
            break

    # پایان زمان
    # برگرداندن بهترین
    best_idx=0
    best_sim=0
    for i,indiv in enumerate(population):
        sq_ = decode_chromosome(indiv,pairs,length_db)
        sim_ = similarity_db(fold_rna(sq_), target_db)
        if sim_>best_sim:
            best_sim=sim_
            best_idx=i
    best_seq = decode_chromosome(population[best_idx], pairs, length_db)
    best_pred = fold_rna(best_seq)
    return (best_seq, best_pred)


#############################################################################
# eM2dRNAs ساده (بدون تغییر)
#############################################################################

def eM2dRNAs(target_db: str, guide: List[int], max_time: float) -> Tuple[str,str]:
    seq, pred_db = m2dRNAs(target_db, max_time)
    return seq, pred_db

#############################################################################
# ES (ساده)
#############################################################################

def es_em2dRNAs(target_db: str, total_time: float) -> Tuple[str,str]:
    # اینجا هم مشابه قبل
    start_time = time.time()
    pairs = parse_dot_bracket(target_db)
    es_budget = total_time/2.0

    def greedy_guide():
        g = [0]*len(pairs)
        for i in range(len(g)):
            if i%10==0:
                g[i] =1
        return g

    x = greedy_guide()
    Xx, Sx = eM2dRNAs(target_db, x, TIME_PER_CALL_ES)
    best_sim = similarity_db(Sx, target_db)
    pm_es = MUT_PROB_ES
    stagnation=0
    while True:
        elapsed = time.time()-start_time
        if elapsed>= es_budget:
            break
        if best_sim>=1.0:
            break

        def mutate_guide(guide, pm):
            new_g= guide[:]
            for i in range(len(new_g)):
                if random.random()<pm:
                    new_g[i] = 1-new_g[i]
            return new_g

        y = mutate_guide(x, pm_es)
        Xy,Sy= eM2dRNAs(target_db, y, TIME_PER_CALL_ES)
        sim_y = similarity_db(Sy, target_db)
        if sim_y>best_sim:
            x=y
            Xx,Sx = Xy,Sy
            best_sim=sim_y
            pm_es = MUT_PROB_ES
            stagnation=0
        else:
            stagnation+=1
            if stagnation>=STAGNATION_GEN_ES:
                pm_es*=1.2
                stagnation=0

    used= time.time()-start_time
    remain = total_time-used
    if remain>0 and best_sim<1.0:
        XX,SS = eM2dRNAs(target_db, x, remain)
        sim_ = similarity_db(SS, target_db)
        if sim_>best_sim:
            Xx, Sx= XX,SS
            best_sim=sim_
    return (Xx, Sx)

#############################################################################
# خواندن فایل دیتاست
#############################################################################

def read_eterna_dataset(filename: str):
    puzzles=[]
    with open(filename,'r') as f:
        for line in f:
            line=line.strip()
            if not line or line.startswith("Puzzle #"):
                continue
            fields= line.split('\t')
            if len(fields)<5:
                continue
            puzzle_id = fields[0].strip()
            puzzle_name= fields[1].strip()
            secondary_structure= fields[4].strip()
            puzzles.append((puzzle_id, puzzle_name, secondary_structure))
    return puzzles

#############################################################################
# main
#############################################################################

def main():
    random.seed(0)
    dataset_file= "/content/drive/MyDrive/Colab Notebooks/eterna100_vienna2.txt"
    puzzles= read_eterna_dataset(dataset_file)
    print(f"Read {len(puzzles)} puzzles from {dataset_file}")

    # مدت اجرا برای هر پازل (مثلاً 600 ثانیه)
    time_per_puzzle = 600.0

    # بسته به سلیقه می‌توانید زیرمجموعه یا همه را اجرا کنید
    # مثلا 10 تا پازل اول:
    #puzzles = puzzles[30:40]

    for idx, (pid, pname, pdb) in enumerate(puzzles, start=1):
        print(f"\nSolving puzzle #{pid} ({pname}) ...")
        seq, pred = es_em2dRNAs(pdb, time_per_puzzle)
        sim = similarity_db(pred, pdb)
        print(f"  Found seq = {seq}")
        print(f"  Pred DB   = {pred}")
        print(f"  Similarity= {sim:.4f}")

if __name__=="__main__":
    main()


ModuleNotFoundError: No module named 'RNA'